In [ ]:
#ВСЕ ТРИ РЕЖИМА контрастная бинаризация

import numpy as np
import matplotlib.pyplot as plt
import os
from pathlib import Path
from PIL import Image
from tqdm import tqdm
import imageio.v2 as imageio  # Используем imageio.v2 для совместимости
from io import BytesIO
from skimage.morphology import footprint_rectangle

from skimage.filters import gaussian, threshold_local
from scipy.ndimage import binary_closing, binary_opening

from scipy.signal import convolve2d
from skimage.morphology import skeletonize
from skimage.filters import threshold_local

import networkx as nx
from scipy.ndimage import convolve
from sklearn.neighbors import KDTree
from scipy import ndimage
from skimage import morphology
from skimage.filters import threshold_otsu

class ReactionDiffusionModel:
    def __init__(self, mode='stripes', with_animation=False):

        self.mode = mode
        self.with_animation = with_animation  # Флаг для анимации

        #ЗАДАЙТЕ!
        self.N = 200 # Размер сетки NxN

        self.treal = 20 if mode != 'doubling' else 27
        self.dt = 1e-7 # шаг по времени
        self.Lreal=2 #моделируемая область (в см)

        self.k = 0.65 # Параметр насыщения
        self.c = 0.03    # Скорость деградации активатора
        self.e = 0.06     # Скорость деградации ингибитора
        self.S =0
              # Источник ингибитора
        self.S_increment = 0.006 # Величина добавки к источнику

        self.U0=1.1
        self.V0=2

        self.U_fl=0.02
        self.V_fl=0.02

        print(f"реальное время (в днях): { self.treal}")
        print(f"шаг по времени: {self.dt}")


        # Параметры  по статье и обезразмеривание
        self.Du_r=0.15e-8
        self.Dv_r=30e-8
        self.D0=15e-8

        self.Du = self.Du_r/self.D0    # D_Ur/D0, обезразмериваем
        self.Dv = self.Dv_r/self.D0
        print(f"диффузия активатора в модели: {self.Du}")
        print(f"диффузия ингибитора в модели: {self.Dv}")

        #геометрия

        self.L0=4 # коэффицент обезразмеривания(в см)
        self.L=self.Lreal/self.L0
        #self.L=1
        self.dx = self.L / self.N

        print(f"размер моделируемой области (в см): {self.L*self.L0}")
        print(f"размер сетки: {self.L}")
        print(f"обезразмеривание по L0 : {self.L0}")
        print(f"шаг сетки: {self.dx}")



        #self.Preal = 40.0 #(в мкм)
        # Параметры времени
        self.Tc=3600
        self.t0 = (self.L0**2)/self.D0
        print(f"характерное время: {self.t0}")
        self.tmod = self.treal*24*3600/(self.t0)
        print(f"время в модели: {self.tmod}")


        # Параметры сетки
        self.dx_real = self.L0*10000* self.dx # перевод сантиметров в микрометры
        # print(f"размер хорошо просматриваемых структур (в мкм) в модели { 2*self.dx_real}")
        print(f"количество узлов сетки { self.N}")
        #self.total_steps = 800000
        self.total_steps = int(self.tmod/self.dt)
        print(f"количество шагов: {self.total_steps}")
        self.gamma = self.t0/self.Tc
        #self.gamma = 15000 # Масштабный параметр (исходное значение)
        print(f"Масштабный параметр gamma: {self.gamma}")

        # Настройки для разных режимов
        if self.mode == 'spots':
            self.S = 2
            pass  # Будем увеличивать S в процессе моделирования
        elif self.mode == 'doubling':
            self.gamma = self.gamma*2  # Увеличиваем gamma для удвоения полос


        np.random.seed(3425)
        self.U = self.U0 * np.ones((self.N, self.N))  # Активатор
        self.V = self.V0 * np.ones((self.N, self.N))    # Ингибитор

        # self.U +=self.U_fl * self.U0 * np.random.random(size=(self.N, self.N))  # +2% шум положительный, во всех ячейках
        # self.V += self.V_fl* self.V0 * np.random.random(size=(self.N, self.N))
        self.U = self.U0 + self.U_fl  * self.U0 * (2 * np.random.random((self.N, self.N)) - 1)
        self.V = self.V0 + self.V_fl * self.V0 * (2 * np.random.random((self.N, self.N)) - 1)


        self.step_count = 0

        # Для анимации
        self.frame_files = []  # Будем хранить пути к временным файлам
        self.partial_gif_counter = 0

    # Вычисление лапласиана с граничными условиями Неймана
    def laplacian(self, Z):
        Ztop = Z[0:-2, 1:-1]
        Zleft = Z[1:-1, 0:-2]
        Zbottom = Z[2:, 1:-1]
        Zright = Z[1:-1, 2:]
        Zcenter = Z[1:-1, 1:-1]
        numerator = Ztop + Zleft + Zbottom + Zright - 4*Zcenter
        return numerator / (self.dx**2 + 1e-10)

    # Вычисление нелинейных реакционных членов
    def reaction_terms(self, U, V):
        U_sq = U[1:-1,1:-1]**2
        V_safe = np.clip(V[1:-1,1:-1], 1e-10, None)
        f = self.gamma * (U_sq / ((1 + self.k*U_sq)*V_safe) - self.c*U[1:-1,1:-1])
        g = self.gamma * (U_sq - self.e*V[1:-1,1:-1] + self.S)
        return f, g



    # Метод Рунге-Кутты 2-го порядка (модифицированный Эйлер)
    def rk2_step(self):
        # Шаг 1: Предиктор (k1) — полный шаг dt
        f1, g1 = self.reaction_terms(self.U, self.V)
        lap_U1 = self.laplacian(self.U)
        lap_V1 = self.laplacian(self.V)

        # Временные U1, V1 (k1)
        U1 = self.U.copy()
        V1 = self.V.copy()
        U1[1:-1, 1:-1] += self.dt * (self.Du * lap_U1 + f1)
        V1[1:-1, 1:-1] += self.dt * (self.Dv * lap_V1 + g1)

        # Граничные условия для U1, V1
        for Z in [U1, V1]:
            Z[0, :] = Z[1, :]; Z[-1, :] = Z[-2, :]
            Z[:, 0] = Z[:, 1]; Z[:, -1] = Z[:, -2]

        # Шаг 2: Корректор (k2) — усреднение k1 и k2
        f2, g2 = self.reaction_terms(U1, V1)
        lap_U2 = self.laplacian(U1)
        lap_V2 = self.laplacian(V1)

        # Итоговое обновление: U += 0.5*dt*(k1 + k2)
        self.U[1:-1, 1:-1] += 0.5 * self.dt * (self.Du * (lap_U1 + lap_U2) + (f1 + f2))
        self.V[1:-1, 1:-1] += 0.5 * self.dt * (self.Dv * (lap_V1 + lap_V2) + (g1 + g2))

        # Граничные условия для новых U, V
        for Z in [self.U, self.V]:
            Z[0, :] = Z[1, :]; Z[-1, :] = Z[-2, :]
            Z[:, 0] = Z[:, 1]; Z[:, -1] = Z[:, -2]

        self.step_count += 1

    # Улучшенный метод контрастирования с тремя режимами бинаризации и сглаживанием краёв
    def _enhance_contrast(self, data):
        # 1. Предварительная обработка (общая для всех режимов)
        filtered = ndimage.median_filter(data, size=3)
        p1, p99 = np.percentile(filtered, [1, 99])
        normalized = (filtered - p1) / (p99 - p1 + 1e-10)
        normalized = np.clip(normalized, 0, 1)

        # 2. Режим-специфичная бинаризация
        if self.mode == 'stripes':
            # Режим полос - широкие гладкие черные полосы
            threshold = 0.5
            binary = normalized > threshold

            # Морфологические операции для сглаживания
            binary = morphology.binary_closing(binary, footprint=morphology.disk(2))
            binary = morphology.binary_opening(binary, footprint=morphology.disk(2))

            # Удаление мелких объектов и сглаживание
            binary = morphology.remove_small_objects(~binary, min_size=16)
            binary = ~binary

            # Дополнительное сглаживание краёв (альтернатива uniform_filter)
            binary = ndimage.gaussian_filter(binary.astype(float), sigma=0.8) > 0.5

        elif self.mode == 'spots':
            # Режим пятен
            threshold = 0.5
            binary = normalized > threshold

            # Сглаживание перед удалением мелких объектов
            binary = morphology.binary_opening(binary, footprint=morphology.disk(1))
            binary = ndimage.gaussian_filter(binary.astype(float), sigma=0.7) > 0.5

            binary = morphology.remove_small_objects(~binary, min_size=8)
            binary = ~binary

        elif self.mode == 'doubling':
            # Режим узких полос
            threshold = 0.5
            binary = normalized > threshold

            # Последовательное сглаживание
            binary = morphology.binary_closing(binary, footprint=morphology.disk(1))
            binary = morphology.binary_opening(binary, footprint=morphology.disk(1))
            binary = morphology.remove_small_objects(~binary, min_size=4)
            binary = ~binary

            # Финальное сглаживание
            binary = ndimage.gaussian_filter(binary.astype(float), sigma=0.5) > 0.5

        # 3. Постобработка (общая для всех режимов)
        binary = morphology.remove_small_holes(binary, area_threshold=8)
        return binary


    def show_state(self, step):
        current_time_days = step * self.dt * self.t0 / (24 * 3600)  # Пересчет шагов в дни

        plt.figure(figsize=(12,8), dpi=100)

        # Применяем бинаризацию
        U_binary = self._enhance_contrast(self.U)
        V_binary = self._enhance_contrast(self.V)
        UV_binary =self._enhance_contrast(self.U - self.V)

        plt.subplot(2,3,1)
        plt.imshow(U_binary, cmap='binary', interpolation='none')
        plt.title(f'Бинарный активатор (время: {current_time_days:.2f} дней)')

        plt.subplot(2,3,2)
        plt.imshow(self.U, cmap=plt.cm.binary, interpolation='bilinear')
        plt.title(f'Активатор U (время: {current_time_days:.2f} дней)')
        plt.colorbar()

        plt.subplot(2,3,3)
        plt.imshow(V_binary, cmap=plt.cm.binary, interpolation='none')
        plt.title(f'Бинарный ингибитор (время: {current_time_days:.2f} дней)')

        plt.subplot(2,3,4)
        plt.imshow(UV_binary, cmap=plt.cm.binary, interpolation='none')
        plt.title(f'U-V система (время: {current_time_days:.2f} дней)')

        plt.subplot(2,3,(5,6))
        center = self.N // 2
        U_norm = self.U[center,:] / np.max(self.U)
        V_norm = self.V[center,:] / np.max(self.V)
        plt.plot(U_norm, label='Норм. активатор (U)')
        plt.plot(V_norm, label='Норм. ингибитор (V)')
        plt.legend()
        plt.title(f'Профили концентраций по центру (время: {current_time_days:.2f} дней)')
        plt.xlabel('Позиция')
        plt.ylabel('Нормированная концентрация')

        plt.tight_layout()
        plt.show()


    # Сохранение кадра с оптимальной бинаризацией
    def _save_frame_to_temp_file(self, step, output_dir):
        current_time_days = step * self.dt * self.t0 / (24 * 3600)  # Пересчет шагов в дни

        Path(output_dir).mkdir(exist_ok=True)

        U_binary = self._enhance_contrast(self.U)
        V_binary = self._enhance_contrast(self.V)
        UV_binary = self._enhance_contrast(self.U - self.V)

        fig = plt.figure(figsize=(12, 8))

        # Первый ряд
        plt.subplot(2, 3, 1)
        plt.imshow(U_binary, cmap='binary', interpolation='none')
        plt.title(f'Активатор (время: {current_time_days:.2f} дней)')

        plt.subplot(2, 3, 2)
        plt.imshow(self.U, cmap='binary', interpolation='bilinear')
        plt.title('Оригинал U')

        plt.subplot(2, 3, 3)
        plt.imshow(UV_binary, cmap='binary', interpolation='none')
        plt.title('U-V система')

        # Второй ряд
        plt.subplot(2, 3, 4)
        plt.imshow(V_binary, cmap='binary', interpolation='none')
        plt.title('Бинарный ингибитор')

        plt.subplot(2,3,(5,6))
        center = self.N // 2
        U_norm = self.U[center,:] / np.max(self.U)
        V_norm = self.V[center,:] / np.max(self.V)
        plt.plot(U_norm, label='Норм. активатор (U)')
        plt.plot(V_norm, label='Норм. ингибитор (V)')
        plt.legend()
        plt.title(f'Профили концентраций по центру (время: {current_time_days:.2f} дней)')
        plt.xlabel('Позиция')
        plt.ylabel('Нормированная концентрация')

        plt.tight_layout()
        frame_path = os.path.join(output_dir, f"frame_{step:06d}.png")
        plt.savefig(frame_path, bbox_inches='tight', dpi=150)
        plt.close()
        return frame_path

    # Создает частичный GIF из временных файлов и очищает их
    def save_partial_gif(self, output_dir):
        if not self.frame_files:
            return

        Path(output_dir).mkdir(exist_ok=True)
        partial_path = os.path.join(output_dir, f'partial_{self.partial_gif_counter}.gif')

        # Читаем все кадры в память и создаем GIF
        with imageio.get_writer(partial_path, mode='I', duration=0.1) as writer:
            for frame_file in self.frame_files:
                image = imageio.imread(frame_file)  # Используем imageio.v2.imread
                writer.append_data(image)
                os.remove(frame_file)  # Удаляем временный файл

        print(f"Создан частичный GIF: {partial_path}")
        self.partial_gif_counter += 1
        self.frame_files = []

    # Собирает финальный GIF из частичных GIF
    def create_final_animation(self, output_dir):
        if self.partial_gif_counter == 0 and not self.frame_files:
            print("Нет данных для создания анимации.")
            return

        final_path = os.path.join(output_dir, f'{self.mode}_animation.gif')

        # Создаем финальный GIF
        with imageio.get_writer(final_path, mode='I', duration=0.1) as writer:
            # Добавляем все частичные GIF
            for i in range(self.partial_gif_counter):
                part_path = os.path.join(output_dir, f'partial_{i}.gif')
                try:
                    with imageio.get_reader(part_path) as reader:
                        for frame in reader:
                            writer.append_data(frame)
                    os.remove(part_path)  # Удаляем частичный GIF после использования
                except FileNotFoundError:
                    print(f"Файл {part_path} не найден, пропускаем.")

            # Добавляем оставшиеся кадры, если они есть
            for frame_file in self.frame_files:
                try:
                    image = imageio.imread(frame_file)
                    writer.append_data(image)
                    os.remove(frame_file)
                except FileNotFoundError:
                    print(f"Файл {frame_file} не найден, пропускаем.")

        self.frame_files = []  # Очищаем список кадров
        print(f"Финальная анимация сохранена: {final_path}")


    # Сохранение результатов с максимальной четкостью
    def save_final_patterns(self, output_dir):
        Path(output_dir).mkdir(exist_ok=True)

        # Применяем улучшенную бинаризацию
        U_binary =  self._enhance_contrast(self.U)
        V_binary = self._enhance_contrast(self.V)
        UV_binary = self._enhance_contrast(self.U - self.V)

        # Сохраняем все варианты изображений
        plt.imsave(os.path.join(output_dir, f'final_U_binary_{self.mode}.png'),
                  U_binary, cmap='binary', dpi=300)

        plt.imsave(os.path.join(output_dir, f'final_V_binary_{self.mode}.png'),
                  V_binary, cmap='binary', dpi=300)

        plt.imsave(os.path.join(output_dir, f'final_UV_binary_{self.mode}.png'),
                  UV_binary, cmap='binary', dpi=300)

        # Оригиналы в градациях серого
        plt.imsave(os.path.join(output_dir, f'final_U_original_{self.mode}.png'),
                  self.U, cmap='binary', dpi=300)

        plt.imsave(os.path.join(output_dir, f'final_V_original_{self.mode}.png'),
                  self.V, cmap='binary', dpi=300)

        # Дополнительно: сохраняем данные в numpy формате для последующего анализа
        np.savez(os.path.join(output_dir, f'final_data_{self.mode}.npz'),
                U=self.U, V=self.V, U_binary=U_binary)


    # Основной цикл моделирования
    def run_simulation(self, output_dir="results",
                      display_interval=1000,
                      gif_save_interval=10000,
                      full_display_interval=10000):
        Path(output_dir).mkdir(exist_ok=True)

        print(f"Начальное состояние системы (режим: {self.mode}):")
        self.show_state(0)
        if self.with_animation:
            self.frame_files.append(self._save_frame_to_temp_file(0, output_dir))

        # =============================================
        # Проверки перед началом моделирования
        # =============================================

        print("\nПроверка условий устойчивости и сходимости:")

        # 1. Проверка условий Тьюринга
        print("\n1. Проверка условий Тьюринга:")



        # Используем начальные значения U0 и V0 для проверки
        U0 = self.U0
        V0 = self.V0

        f_u = self.gamma * ((2*U0*(1 + self.k*U0**2)-2*self.k*U0**3)/(V0*(1 + self.k*U0**2)**2) - self.c)
        f_v = -self.gamma * (U0**2/((1 + self.k*U0**2)*V0**2))
        g_u = 2 * self.gamma * U0
        g_v = -self.gamma * self.e

        print(f"f_u = {f_u:.4f}, f_v = {f_v:.4f}")
        print(f"g_u = {g_u:.4f}, g_v = {g_v:.4f}")

        # Первое условие Тьюринга: f_u + g_v < 0
        cond1 = f_u + g_v
        print(f"f_u + g_v = {cond1:.4f} (должно быть < 0): {'выполнено' if cond1 < 0 else 'НЕ ВЫПОЛНЕНО!'}")

        # Второе условие Тьюринга: f_u*g_v - f_v*g_u > 0
        cond2 = f_u*g_v - f_v*g_u
        print(f"f_u*g_v - f_v*g_u = {cond2:.4f} (должно быть > 0): {'выполнено' if cond2 > 0 else 'НЕ ВЫПОЛНЕНО!'}")

        # Третье условие Тьюринга: Dv*f_u + Du*g_v > 2*sqrt(Du*Dv*(f_u*g_v - f_v*g_u))
        left = self.Dv*f_u + self.Du*g_v
        right = 2*np.sqrt(self.Du*self.Dv*(f_u*g_v - f_v*g_u))
        cond3 = left - right
        print(f"cond3 = {cond3:.4f}")
        print(f"(Dv*f_u + Du*g_v) = {left:.4f} > 2*sqrt(Du*Dv*(f_u*g_v - f_v*g_u)))= {right:.4f}: {'выполнено' if cond3 > 0 else 'НЕ ВЫПОЛНЕНО!'}")

        # 2. Проверка числа Куранта (условие устойчивости)
        print("\n2. Проверка числа Куранта (устойчивость):")

        # Для двумерного случая максимальный шаг по времени:

        dt_max = (self.dx**2) / (4 * max(self.Du, self.Dv))
        courant = self.dt / dt_max
        print(f"Текущий dt = {self.dt:.2e}, максимально допустимый dt = {dt_max:.2e}")
        print(f"Число Куранта = {courant:.4f} (должно быть < 1): {'выполнено' if courant < 1 else 'НЕ ВЫПОЛНЕНО!'}")

        # 3. Проверка на сходимость
        print("\n3. Проверка на сходимость (эмпирическое правило):")

        #dt должен быть  меньше характерного времени реакции
        # Характерное время реакции ~ c/Tc
        reaction_time = self.c / (self.Tc)
        ratio = self.dt / reaction_time
        print(f"Характерное время реакции = {reaction_time:.2e}")
        print(f"Отношение dt/τ_реакции = {ratio:.4f} (рекомендуется < 0.1): {'OK' if ratio < 0.1 else 'ВНИМАНИЕ: возможно недостаточно малый шаг!'}")

        #input("\nПроверки завершены. Нажмите Enter для продолжения моделирования...")

        # =============================================
        # Основной цикл моделирования
        # =============================================


        for step in tqdm(range(1, self.total_steps + 1), desc="Прогресс моделирования"):
            # В режиме 'spots' увеличиваем S каждые 1000 шагов
            if self.mode == 'spots' and step % (self.total_steps/1000) == 0 and step > 0:
                self.S += self.S_increment
                #print(f"\nШаг {step}: увеличено S до {self.S:.6f}")

            self.rk2_step()

            if step % display_interval == 0:
                if step % full_display_interval == 0:
                    self.show_state(step)

                if self.with_animation:
                    self.frame_files.append(self._save_frame_to_temp_file(step, output_dir))

                    if step % gif_save_interval == 0:
                        self.save_partial_gif(output_dir)

        print("\nМоделирование завершено!")
        print(f"Финальное значение S: {self.S:.6f}")

        self.show_state(self.total_steps)

        if self.with_animation:
            self.create_final_animation(output_dir)

        # Сохраняем финальные паттерны (всегда)
        self.save_final_patterns(output_dir)


# Запуск всех трех режимов последовательно и вывод результатов
def run_all_modes():
    modes = ['stripes', 'spots', 'doubling']
    final_U_images = []
    final_U_binary_images = []

    for mode in modes:
        print(f"\n{'='*50}")
        print(f"Запуск моделирования в режиме: {mode}")
        print(f"{'='*50}\n")

        # Создаем модель с текущим режимом и анимацией
        model = ReactionDiffusionModel(mode=mode, with_animation=True)

        # Запускаем моделирование с настройками по умолчанию
        model.run_simulation(
            output_dir=f"results_{mode}",
            display_interval=1000,
            gif_save_interval=10000,
            full_display_interval=10000
        )

        # Сохраняем финальные изображения для отображения
        final_U_images.append(model.U.copy())
        final_U_binary_images.append(model._enhance_contrast(model.U))

    # Отображаем все финальные изображения
    plt.figure(figsize=(12, 8))

    # Оригинальные изображения
    for i, (img, mode) in enumerate(zip(final_U_images, modes)):
        plt.subplot(2, 3, i+1)
        plt.imshow(img, cmap='binary')
        plt.title(f'Оригинал U ({mode})')
        plt.axis('off')

    # Бинаризованные изображения
    for i, (img, mode) in enumerate(zip(final_U_binary_images, modes)):
        plt.subplot(2, 3, i+4)
        plt.imshow(img, cmap='binary')
        plt.title(f'Бинаризованный U ({mode})')
        plt.axis('off')

    plt.tight_layout()
    plt.show()

if __name__ == "__main__":
    run_all_modes()